In [127]:
#pip install pymongo



Создать базу данных.

Вставить 4 документа по товарам на сайте. Атрибуты:
* название;
* категория — 2 товара из одной категории и 2 товара из другой;
* цена;
* количество товаров на складе.

Рассчитать остаточную стоимость товаров в каждой категории (сумма цены, умноженной на остаток).

Уменьшить количество товара на 1.

Вывести 2 самых дорогих товара.


In [188]:
import pymongo 

In [189]:
# Создадим подключение к базе: 
client = pymongo.MongoClient('localhost', 27017)

# Connect to our database
db = client['goods']

# Fetch our series collection
#series_collection = db['series']

Создать базу данных.

In [201]:
# Посмотрим что подключилось
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'goods')


Данные в MongoDB представлены и хранятся в виде JSON документов . В PyMongo мы используем словари представления документов. В качестве примера, следующий словарь может быть использован для представления докуммнта о товаре:

Вставить 4 документа по товарам на сайте. Атрибуты:
* название;
* категория — 2 товара из одной категории и 2 товара из другой;
* цена;
* количество товаров на складе.

In [221]:
# добавит пакетно все товары через список словарей:
new_goods = [
        {"name": "Prezident",
        "category": "Кофе",
        "price": 1500.0,
        "quantity": 120}, 
        {"name": "Egoist",
        "category": "Кофе",
        "price": 2500.0,
        "quantity": 20},
         {"name": "Assam",
        "category": "Чай",
        "price": 300.0,
        "tags":['чёрный', 'пакетированный'],  # неформализованная схема позволяет произвольно добавить нужные атрибуты
        "quantity": 54},
         {"name": "Lipton",
        "category": "Чай",
        "price": 250.0,
        "quantity": 72}]

result = goods.insert_many(new_goods)



Убедимся что всё записано

In [222]:
for good in goods.find():
    display(good)


{'_id': ObjectId('640626e555423ab1de751916'),
 'name': 'Prezident',
 'category': 'Кофе',
 'price': 1500.0,
 'quantity': 120}

{'_id': ObjectId('640626e555423ab1de751917'),
 'name': 'Egoist',
 'category': 'Кофе',
 'price': 2500.0,
 'quantity': 20}

{'_id': ObjectId('640626e555423ab1de751918'),
 'name': 'Assam',
 'category': 'Чай',
 'price': 300.0,
 'tags': ['чёрный', 'пакетированный'],
 'quantity': 54}

{'_id': ObjectId('640626e555423ab1de751919'),
 'name': 'Lipton',
 'category': 'Чай',
 'price': 250.0,
 'quantity': 72}

Рассчитать остаточную стоимость товаров в каждой категории (сумма цены, умноженной на остаток).

In [223]:
pipeline = [{ 
# Проводим группировку:
    "$group":  

        {"_id" : "$category",  
# Обсчитываем категорию:
          "По категории всего": { "$sum": { "$multiply": [ "$price", "$quantity" ] } }
         }} 

    ]

agg_result= goods.aggregate(pipeline) 


# Выводим результат:
for goods_group in agg_result: 
   print(goods_group)


{'_id': 'Кофе', 'По категории всего': 230000.0}
{'_id': 'Чай', 'По категории всего': 34200.0}


In [ ]:
https://www.mongodb.com/docs/manual/core/aggregation-pipeline/

db.orders.aggregate( [
   // Stage 1: Filter pizza order documents by date range
   {
      $match:
      {
         "date": { $gte: new ISODate( "2020-01-30" ), $lt: new ISODate( "2022-01-30" ) }
      }
   },
   // Stage 2: Group remaining documents by date and calculate results
   {
      $group:
      {
         _id: { $dateToString: { format: "%Y-%m-%d", date: "$date" } },
         totalOrderValue: { $sum: { $multiply: [ "$price", "$quantity" ] } },
         averageOrderQuantity: { $avg: "$quantity" }
      }
   },
   // Stage 3: Sort documents by totalOrderValue in descending order
   {
      $sort: { totalOrderValue: -1 }
   }
 ] )

**Вывести 2 самых дорогих товара:**

In [224]:
cursor = goods.find().sort("price", pymongo.DESCENDING).limit(2)  
for doc_ in cursor:
    print(doc_)


{'_id': ObjectId('640626e555423ab1de751917'), 'name': 'Egoist', 'category': 'Кофе', 'price': 2500.0, 'quantity': 20}
{'_id': ObjectId('640626e555423ab1de751916'), 'name': 'Prezident', 'category': 'Кофе', 'price': 1500.0, 'quantity': 120}


**Уменьшить количество товара на 1 единицу**

In [225]:
# Обновить для чая Assam
filter_ = {'name':'Assam' }
 
# Уменьшить количество на одну единицу
newvalue = { "$inc": { 'quantity': -1 } }

goods.update_many(filter_, newvalue)
cursor = goods.find(filter_)

for record in cursor:
    print(record)

{'_id': ObjectId('640626e555423ab1de751918'), 'name': 'Assam', 'category': 'Чай', 'price': 300.0, 'tags': ['чёрный', 'пакетированный'], 'quantity': 53}


In [226]:
# Обновить для всего
filter_ = {} # нужен пустой фильтр !!!! 
 
# Уменьшить количество на одну единицу
newvalue = { "$inc": { 'quantity': -1 } }

goods.update_many(filter_, newvalue)
cursor = goods.find(filter_)

for record in cursor:
    print(record)

{'_id': ObjectId('640626e555423ab1de751916'), 'name': 'Prezident', 'category': 'Кофе', 'price': 1500.0, 'quantity': 119}
{'_id': ObjectId('640626e555423ab1de751917'), 'name': 'Egoist', 'category': 'Кофе', 'price': 2500.0, 'quantity': 19}
{'_id': ObjectId('640626e555423ab1de751918'), 'name': 'Assam', 'category': 'Чай', 'price': 300.0, 'tags': ['чёрный', 'пакетированный'], 'quantity': 52}
{'_id': ObjectId('640626e555423ab1de751919'), 'name': 'Lipton', 'category': 'Чай', 'price': 250.0, 'quantity': 71}


In [227]:
# Обновить для категории
filter_ = {'category':'Чай'}
 
# Уменьшить количество на одну единицу
newvalue = { "$inc": { 'quantity': -1 } }

goods.update_many(filter_, newvalue)
cursor = goods.find(filter_)

for record in cursor:
    print(record)

{'_id': ObjectId('640626e555423ab1de751918'), 'name': 'Assam', 'category': 'Чай', 'price': 300.0, 'tags': ['чёрный', 'пакетированный'], 'quantity': 51}
{'_id': ObjectId('640626e555423ab1de751919'), 'name': 'Lipton', 'category': 'Чай', 'price': 250.0, 'quantity': 70}


https://merrick-krg.medium.com/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D0%BA%D1%83%D1%80%D1%81-%D0%BF%D0%BE-mongodb-cab518a8f2fe

In [ ]:
Удалить лишние строки

In [228]:
# Строим фильтр:
'''
filter_ = {'_id':{'$in':['640596cc55423ab1de7518f9',
                         '640596cc55423ab1de7518fa',       
                         '6405f4f955423ab1de75190f', 
                         '6405f4f955423ab1de751910',
                         '640596cc55423ab1de7518f7',  
                         '640596cc55423ab1de7518f8', 
                         '640596cc55423ab1de7518f9'
                        ]}}
'''
filter_ = {'quantity':{'$in':[10,69]}}
    
d = goods.delete_many(filter_)
cursor = goods.find()
print(d.deleted_count, " documents deleted !!")


for record in cursor:
    print(record)

0  documents deleted !!
{'_id': ObjectId('640626e555423ab1de751916'), 'name': 'Prezident', 'category': 'Кофе', 'price': 1500.0, 'quantity': 119}
{'_id': ObjectId('640626e555423ab1de751917'), 'name': 'Egoist', 'category': 'Кофе', 'price': 2500.0, 'quantity': 19}
{'_id': ObjectId('640626e555423ab1de751918'), 'name': 'Assam', 'category': 'Чай', 'price': 300.0, 'tags': ['чёрный', 'пакетированный'], 'quantity': 51}
{'_id': ObjectId('640626e555423ab1de751919'), 'name': 'Lipton', 'category': 'Чай', 'price': 250.0, 'quantity': 70}


In [ ]:
#Удалим кофе
query = {"category": "Кофе"} 
result =goods.delete_many(query)
